In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Fungsi untuk login ke Twitter
def twitter_login(driver, username, password):
    driver.get("https://twitter.com/login")
    time.sleep(5)  # Tunggu halaman login dimuat
    
    # Input username
    username_field = driver.find_element(By.NAME, "text")
    username_field.send_keys(username)
    username_field.send_keys(Keys.RETURN)
    time.sleep(3)
    # Input password
    try:
        password_field = driver.find_element(By.NAME, "password")
        password_field.send_keys(password)
        password_field.send_keys(Keys.RETURN)
        time.sleep(5)  # Tunggu proses login selesai
    except Exception as e:
        print("Gagal menemukan/memasukkan password:", e)
        return False

from selenium.webdriver.common.by import By

# Fungsi untuk melakukan pencarian dan scraping
def scrape_tweets(driver, query):
    # # Buka halaman pencarian Twitter
    # driver.get(f"https://twitter.com/search?q={query}&src=typed_query")
    # time.sleep(5)  # Tunggu hasil pencarian dimuat
    
    # # Langkah tambahan: Pilih tab "Latest"
    # try:
    #     # Cari elemen tab "Latest" berdasarkan atribut href atau data-testid
    #     latest_tab = driver.find_element(By.XPATH, '//a[@href="/search?q={}&src=typed_query&f=live"]'.format(query))
    #     latest_tab.click()
    #     print("Tab 'Latest' berhasil dipilih.")
    #     time.sleep(3)  # Tunggu konten tab "Latest" dimuat
    # except Exception as e:
    #     print(f"Error saat memilih tab 'Latest': {e}")
    
    tweets_data = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    max_scrolls = 1000  # Batas maksimal scroll
    scroll_count = 0  # Penghitung jumlah scroll
    
    while True:
        # Scroll halaman untuk memuat lebih banyak tweet
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Tunggu konten baru dimuat
        
        # Ambil semua tweet yang muncul di halaman
        tweets = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="tweet"]')
        for tweet in tweets:
            try:
                # Ambil teks tweet
                tweet_text = tweet.find_element(By.CSS_SELECTOR, 'div[data-testid="tweetText"]').text
                
                # Ambil gambar (jika ada)
                try:
                    image_url = tweet.find_element(By.CSS_SELECTOR, 'img[alt="Image"]').get_attribute("src")
                except:
                    image_url = None
                
                # Ambil link tweet
                try:
                    tweet_link = tweet.find_element(By.CSS_SELECTOR, 'a[href^="/"]').get_attribute("href")
                except:
                    tweet_link = None
                
                # Simpan data tweet
                tweets_data.append({
                    "tweet": tweet_text,
                    "image_url": image_url,
                    "tweet_link": tweet_link
                })
            except Exception as e:
                print(f"Error saat mengambil tweet: {e}")
        
        # Periksa apakah sudah mencapai akhir halaman
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("Sudah mencapai akhir halaman.")
            break
        
        # Update ketinggian halaman
        last_height = new_height
        
        # Tambahkan penghitung scroll
        scroll_count += 1
        print(f"Scroll ke-{scroll_count} dari maksimal {max_scrolls}...")
        
        # Berhenti jika sudah mencapai batas maksimal scroll
        if scroll_count >= max_scrolls:
            print("Batas maksimal scroll tercapai.")
            break
    
    return tweets_data

In [ ]:
# Konfigurasi WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Login ke Twitter
username = "username"
password = "password"
twitter_login(driver, username, password)

In [26]:
# Pencarian dan scraping
query = "RUU TNI"
tweets = scrape_tweets(driver, query)

# Simpan hasil ke CSV
df = pd.DataFrame(tweets)
df.to_csv("tweets_ruutni_latest_210425.csv", index=False)
print("Data tweet berhasil disimpan ke tweets_ruutni.csv")

# Tutup browser
driver.quit()

Scroll ke-1 dari maksimal 1000...
Scroll ke-2 dari maksimal 1000...
Scroll ke-3 dari maksimal 1000...
Scroll ke-4 dari maksimal 1000...
Scroll ke-5 dari maksimal 1000...
Scroll ke-6 dari maksimal 1000...
Scroll ke-7 dari maksimal 1000...
Scroll ke-8 dari maksimal 1000...
Scroll ke-9 dari maksimal 1000...
Scroll ke-10 dari maksimal 1000...
Scroll ke-11 dari maksimal 1000...
Error saat mengambil tweet: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-testid="tweetText"]"}
  (Session info: chrome=135.0.7049.96); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x008F8073+60707]
	GetHandleVerifier [0x008F80B4+60772]
	(No symbol) [0x00720683]
	(No symbol) [0x00768660]
	(No symbol) [0x007689FB]
	(No symbol) [0x0075DD81]
	(No symbol) [0x0078D094]
	(No symbol) [0x0075DCA4]
	(No symbol) [0x0078D2C4]
	(No symbol) [0x007AE